In [ ]:
%pylab inline
%load_ext autoreload
%autoreload 2     # actually reload imports
#%config InlineBackend.figure_format = 'svg'    # use vectorial figures
from smartg_view import semi_polar, smartg_view, plot2dtest
from smartg import Smartg, RoughSurface, LambSurface, FlatSurface, Environment
from profile.profil import *
from luts import MLUT, LUT, Idx, merge, read_lut_hdf, read_mlut_hdf
from water.iop_spm import IOP_SPM
from water.iop_mm import IOP_MM
import ephem
from PAR import Int, Int2, Irr, ReadREPTRAN_bands, dailyPAR, viewPAR, ObsPAR, simulatePAR
import pickle

In [ ]:
fpp_l_l, fsp_l_l, fpp2_l_l, fsp2_l_l, dt_l_l, SZA_l, raa_l = simulatePAR(CALC=False)
# store file lists
#with open('simulations_list_continental.pickle3', 'w') as f:
  #pickle.dump([fpp_l_l, fsp_l_l, fpp2_l_l, fsp2_l_l, SZA_l, raa_l], f)
# recompute date list
fpp_l_l, fsp_l_l, fpp2_l_l, fsp2_l_l, dt_l_l, SZA_l, raa_l = simulatePAR(CALC=False,verbose=True)
# Getting back the file lists:
with open('simulations_list_continental.pickle') as f:
    fpp_l_l, fsp_l_l, fpp2_l_l, fsp2_l_l, SZA_l, raa_l = pickle.load(f)

In [ ]:
fpp_l_l, fsp_l_l, fpp2_l_l, fsp2_l_l, dt_l_l, SZA_l, raa_l=simulatePAR(CALC=False)

In [ ]:
# Getting back the objects:
with open('simulations_list_continental.pickle') as f:
    fpp_l_l, fsp_l_l, fpp2_l_l, fsp2_l_l, SZA_l, raa_l = pickle.load(f)

In [ ]:
ObsPAR(fpp2_l_l, fsp2_l_l, SZA_l, raa_l,verbose=True)

In [ ]:
dailyPAR(fpp_l_l,fsp_l_l,dt_l_l,verbose=True)

In [ ]:
with open('simulations_list_stratocumulus5.pickle') as f:
    fpp_l_l, fsp_l_l, fpp2_l_l, fsp2_l_l, SZA_l, raa_l = pickle.load(f)

In [ ]:
ObsPAR(fpp2_l_l, fsp2_l_l, SZA_l, raa_l, verbose=True)

In [ ]:
dailyPAR(fpp_l_l,fsp_l_l,dt_l_l,verbose=True)

In [ ]:
with open('simulations_list_cirrus.pickle') as f:
    fpp_l_l, fsp_l_l, fpp2_l_l, fsp2_l_l, SZA_l, raa_l = pickle.load(f)

In [ ]:
ObsPAR(fpp2_l_l, fsp2_l_l, SZA_l, raa_l)

In [ ]:
dailyPAR(fpp_l_l,fsp_l_l,dt_l_l,verbose=False)

In [ ]:
with open('simulations_list_stratocumulus50.pickle') as f:
    fpp_l_l, fsp_l_l, fpp2_l_l, fsp2_l_l, SZA_l, raa_l = pickle.load(f)

In [ ]:
ObsPAR(fpp2_l_l, fsp2_l_l, SZA_l, raa_l,verbose=False)

In [ ]:
dailyPAR(fpp_l_l,fsp_l_l,dt_l_l,verbose=False)

In [ ]:
LMIN=400. # nm
LMAX=700. # nm
SAMPLING=1
PFRES=50. # spectral resolution for phase function (nm)
grid='100[20]20[5]10[1]0' # altitude grid for scattering and absorption coefficients
o3=300. # DU
wref=862. # wavelength for AOT and COT reference
aot=0.1
aermodel_l=['maritime_clean','continental_average']
aermodel=aermodel_l[1]
chl=0.1 # mg/l
ws=5. # m/s
aer = AeroOPAC(aermodel, aot, wref)
repname='reptran_solar_coarse' # k distribution
surf=RoughSurface(SUR=3, WIND=ws, NH2O=1.34)
wi,wb,we,ex,dl,bands = ReadREPTRAN_bands(repname,LMIN=LMIN,LMAX=LMAX,SAMPLING=SAMPLING,FULL=True)
pfwav=np.linspace(LMIN, LMAX, num=(LMAX-LMIN)/PFRES+1, endpoint=True)
water=IOP_MM(chl,NANG=7201,pfwav=pfwav)
atm=Profile('afglmw',O3=o3,pfwav=pfwav
             #,cloud=CloudOPAC('strato',[('wc.sol.mie', 1, 10. , 9., 10.)], 5, 862.)
             ,aer=aer 
           )
             


In [ ]:
job = Smartg('SMART-G-PP', THVDEG=30, wl=bands, NBPHOTONS=1e8,
                    atm=atm,NBLOOP=5000, OUTPUT_LAYERS=3,env=Environment(ENV=1,ENV_SIZE=10.,X0=0.,ALB=0.5),
                    surf=surf, water=water,NFAER=10000,NFOCE=10000,
                    overwrite=True)

In [ ]:
job.view()

In [ ]:
# multispectral
chl=1 # mg/l
ws=5. # m/s
o3=300. # DU
SPM=10. # mg/l
LMIN=765. # nm
LMAX=768. # nm
SAMPLING=1
PFRES=20. # spectral resolution for phase function (nm)
pfwav=np.linspace(LMIN, LMAX, num=(LMAX-LMIN)/PFRES+1, endpoint=True)
surf=RoughSurface(SUR=1, WIND=ws, NH2O=1.34)
#water=IOP_MM(chl,NANG=7201,pfwav=pfwav)
water=IOP_SPM(SPM,NANG=7201,pfwav=pfwav)
pro = Profile('afglmw',
              grid='100[20]20[5]10[1]0', # altitude grid for scattering and absorption coefficients
              #pfgrid=[100, 20, 10., 5., 0.],   # optional, otherwise use a single band 100-0
              pfwav=pfwav, # optional, otherwise phase functions are calculated at all bands
              #aer=AeroOPAC('continental_average', 0.1, 862.),O3=o3,
              verbose=False)
repname='reptran_solar_fine' # k distribution
wi,wb,we,ex,dl,bands = ReadREPTRAN_bands(repname,LMIN=LMIN,LMAX=LMAX,SAMPLING=SAMPLING,FULL=True)
jobo2 = Smartg('SMART-G-PP', THVDEG=30., wl=bands, NBPHOTONS=1e9,
                    atm=pro,NBLOOP=5000, OUTPUT_LAYERS=3,
                    surf=surf, NFAER=10000,NFOCE=10000,env=Environment(ENV=0,ENV_SIZE=10.,X0=0.,ALB=0.5),
                    #water=water,
                    overwrite=True,cmdfile='/home/did/RTC/SMART-G/zozo.txt')

In [ ]:
jobo2_sp = Smartg('SMART-G-SP', THVDEG=30, wl=bands, NBPHOTONS=1e9,
                    atm=pro,NBLOOP=5000, OUTPUT_LAYERS=3,
                    surf=surf, NFAER=10000,NFOCE=10000,env=Environment(ENV=0,ENV_SIZE=10.,X0=0.,ALB=0.5),
                    water=water,
                    overwrite=True,cmdfile='/home/did/RTC/SMART-G/zozo.txt')

In [ ]:
from luts import MLUT
M=jobo2.read()
MM_l=[]
for k in range(len(M.luts)):
    MM_tmp = M.luts[k]
    MM=LUT(MM_tmp.data,axes=[wi.data,MM_tmp.axes[1],MM_tmp.axes[2]],desc=MM_tmp.desc,
           names=['Wavelengths',MM_tmp.names[1],MM_tmp.names[2]],attrs=MM_tmp.attrs)
    MM_l.append(MM)
M2 = MLUT(MM_l)

In [ ]:
def SpecInt(wi, wb, ex, we, dl, M=None, field=None, lim=[400.,700.]):
    '''
    all input vectors have same length, coming from REPTRAN
    wi : input wavelengths of internal bands (nm)
    wb : input wavelengths of bands (nm)
    ex : input extra-terrestrial irradiances at internal bands (W m-2 nm-1)
    we : input weights of internal bands
    dl : input bandwidths of bands (nm)
    M  : optional LUT or MLUT with 3D (lambda,phi,theta) or 1D (lambda) radiative field to spectrally integrate
    lim: spectral boundaries for integration
    
    returns
    spectrally integrated intensity
    '''
    ok=np.where((wb.data >=lim[0]) & (wb.data <lim[1]))[0]
    if (M != None) :
        if (field != None) :
            L = M[field]
            tab = L.data
        else :
            tab = M.data

        if tab.ndim == 3 :
            R = np.rollaxis(tab,0,3)
            E = LUT(sum(R[:,:,ok] * we.data[ok] * dl.data[ok], axis=2), \
                axes=[L.axes[1], L.axes[2]], desc=L.desc, \
                names=[L.names[1], L.names[2]], attrs={'LAMBDA':lim[0]})
        else:
            E = sum(tab[ok] * we.data[ok] * dl.data[ok])
    else:
        E = sum(we.data[ok] * dl.data[ok])

    norm = sum(we.data[ok] * dl.data[ok])
    Eavg = E/norm

    return E, Eavg


def SpecInt2(wi, wb, ex, we, dl, M=None, field=None, Irradiance=False):
    Eavg=[] 
    wu = np.unique(wb.data)
    for linf in wu:
    #for linf in np.linspace(lim[0],lim[1]-DL,endpoint=True,num=(lim[1]-lim[0])/DL):
        E1,E2 = SpecInt(wi, wb, ex, we, dl, M=M, field=field,lim=[linf,linf+1e-5])
        if Irradiance :
            Eavg.append(Irr(E2))
        else:
            Eavg.append(E2)
    Mavg = merge(Eavg, ['LAMBDA'])
    return  Mavg


In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt
M=M2['I_up (TOA)']
x=M.axes[1]
y=M.axes[2]
z=M.axes[0]
X, Y = np.meshgrid(x,y)
Z = M.sub[200,:,:].data
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.contourf(X,Y,Z,zdir='z',levels=.1*z)
#x = np.linspace(0, 1, 100)
#X, Y = np.meshgrid(x, x)
#Z = np.sin(X)*np.sin(Y)

levels = np.linspace(-1, 1, 40)
#ax.contourf(X,Y,Z)

#ax.contourf(X, Y, .1*np.sin(3*X)*np.sin(5*Y), zdir='z', levels=.1*levels)
#ax.contourf(X, Y, 3+.1*np.sin(5*X)*np.sin(8*Y), zdir='z', levels=3+.1*levels)
#ax.contourf(X, Y, 7+.1*np.sin(7*X)*np.sin(3*Y), zdir='z', levels=7+.1*levels)

ax.legend()
ax.set_xlim3d(0, 180)
ax.set_ylim3d(0, 90)
ax.set_zlim3d(z[0], z[-1])

In [ ]:
def plot2dtest(lut, index=None, vmin=None, vmax=None, rect='211', sub='212', sym=None, swap=False, fig=None):
    '''
    Contour and eventually transect of 2D LUT on a cartesian plot, with
    dimensions (angle, radius)

    lut: 2D look-up table to display
    index: index of the item to transect in the 'angle' dimension
           if None (default), no transect
    vmin, vmax: range of values
                default None: determine min/max from values
    rect: subplot position of the main plot ('111' for example)
    sub: subplot position of the transect
    sym: the transect uses symmetrical axis (boolean)
         if None (default), use symmetry iff axis is 'zenith'
    swap: swap the order of the 2 axes to (radius, angle)
    fig : destination figure. If None (default), create a new figure.
    '''
    from mpl_toolkits.axisartist.grid_finder import FixedLocator, DictFormatter

    #
    # initialization
    #

    assert lut.ndim == 2

    show_sub = index is not None
    if fig is None:
        if show_sub:
            fig = figure(figsize=(4.5, 4.5))
        else:
            fig = figure(figsize=(4.5, 6))

    if swap:
        ax1, ax2 = lut.axes[1], lut.axes[0]
        name1, name2 = lut.names[1], lut.names[0]
        data = np.swapaxes(lut.data, 0, 1)
    else:
        ax1, ax2 = lut.axes[0], lut.axes[1]
        name1, name2 = lut.names[0], lut.names[1]
        data = lut.data

    if vmin is None:
        vmin = np.amin(lut.data)
    if vmax is None:
        vmax = np.amax(lut.data)
    if vmin == vmax:
        vmin -= 0.001
        vmax += 0.001
    if vmin > vmax: vmin, vmax = vmax, vmin

    #
    # semi polar axis
    #
    ax1_ticks = [0, 45, 90, 135, 180]
    if 'azimu' in name1.lower():
        ax1_min, ax1_max = 0., 180.
        ax1_ticks = dict(zip(ax1_ticks, map(str, ax1_ticks)))
        label1 = r'$\phi$'
        ax1_scaled = ax1
    else:
        ax1_min, ax1_max = ax1[0], ax1[-1]
        ax1_ticks = dict(zip(ax1_ticks,
                         map(lambda x: '{:.1f}'.format(x), np.linspace(ax1_min, ax1_max, len(ax1_ticks)))))
        label1 = name1

        # rescale ax1 to (0, 180)
        ax1_scaled = (ax1-ax1_min)/(ax1_max-ax1_min)*180.

    ax2_ticks = [0, 30, 60, 90]
    if 'zenit' in name2.lower():
        ax2_min, ax2_max = 0, 90.
        if sym is None: sym=True
        ax2_ticks = dict(zip(ax2_ticks, map(str, ax2_ticks)))
        label2 = r'$\theta$'
        ax2_scaled = ax2
    else:
        ax2_min, ax2_max = ax2[0], ax2[-1]
        ax2_ticks = dict(zip(ax2_ticks,
                             map(lambda x: '{:.1f}'.format(x), np.linspace(ax2_min, ax2_max, len(ax2_ticks)))))
        if sym is None: sym=False
        label2 = name2

        # rescale to (0, 90)
        ax2_scaled = (ax2-ax2_min)/(ax2_max-ax2_min)*90.

    # 1st axis
    grid_locator1 = FixedLocator(ax1_ticks.keys())
    tick_formatter1 = DictFormatter(ax1_ticks)

    # 2nd axis
    grid_locator2 = FixedLocator(ax2_ticks.keys())
    tick_formatter2 = DictFormatter(ax2_ticks)


    #fig.add_subplot(ax_polar)
    ax_polar = fig.add_subplot(rect)

    #
    # initialize the cartesian axis below the semipolar
    #
    if show_sub:
        ax_cart = fig.add_subplot(sub)
        if sym:
            ax_cart.set_xlim(-ax2_max, ax2_max)
        else:
            ax_cart.set_xlim(ax2_min, ax2_max)
        ax_cart.set_ylim(vmin, vmax)
        ax_cart.grid(True)

    #
    # draw colormesh
    #
    cmap = cm.jet
    cmap.set_under('black')
    cmap.set_over('white')
    cmap.set_bad('0.5') # grey 50%
    r, t = np.meshgrid(ax2_scaled, ax1_scaled)
    masked_data = np.ma.masked_where(np.isnan(data) | np.isinf(data), data)
    im = ax_polar.pcolormesh(t, r, masked_data, cmap=cmap, vmin=vmin, vmax=vmax)

    if show_sub:
        # convert Idx instance to index if necessarry
        if isinstance(index, Idx):
            index = int(round(index.index(ax1)))

        # draw line over colormesh
        vertex0 = np.array([[0,0],[ax1_scaled[index],ax2_max]])
        vertex1 = np.array([[0,0],[ax1_scaled[-1-index],ax2_max]])
        ax_polar.plot(vertex0[:,0],vertex0[:,1], 'w')
        if sym:
            ax_polar.plot(vertex1[:,0],vertex1[:,1],'w--')

        #
        # plot transects
        #
        ax_cart.plot(ax2, data[index,:],'k-')
        if sym:
            ax_cart.plot(-ax2, data[-1-index,:],'k--')

    # add colorbar
    fig.colorbar(im, orientation='horizontal', extend='both', ticks=np.linspace(vmin, vmax, 5))
    if lut.desc is not None:
        ax_polar.set_title(lut.desc, weight='bold', position=(0.15,0.9))


In [ ]:
iqu=0
VMAX=0.3
M=M2['I_up (TOA)']
x=M.axes[1]
y=M.axes[2]
cmap = plt.cm.get_cmap("winter")
cmap.set_under("black")
cmap.set_over("white")
origin = 'lower'
w_tran= 766.
ths_tran=30.
plane_azi=178.

for field in M2.params:
    if (field != 'Numbers of photons') and ('0-' not in field) and ('down' not in field):
        S = SpecInt2(wi, wb, ex, we, dl, M=M2, field=field, Irradiance=False)
        z = S.axes[0]
        Y, Z = np.meshgrid(y,z)
        if iqu==0 : 
            I = S
            desc=field
        if iqu==1 : Ip = S
        #if iqu==2 : 
            #Ip.data=np.sqrt(Ip.data**2 + S.data**2)
            #Ip.desc=desc + 'P'
            
        iqu+=1
        if iqu==3: 
            iqu=0
            fig=figure(figsize=(8,8))
            #plot2dtest(I.sub[:,Idx(plane_azi),:], rect=241, sub=245, swap=True, index=Idx(ths_tran),fig=fig,vmin=0,vmax=VMAX)
            semi_polar(I.sub[:,Idx(plane_azi),:], rect=421, sub=423, swap=True, index=Idx(ths_tran),fig=fig,vmin=0,vmax=VMAX)
            semi_polar(Ip.sub[:,Idx(plane_azi),:], rect=425, sub=427, swap=True, index=Idx(ths_tran),fig=fig,vmin=-0.12,vmax=0)
            #semi_polar(I.sub[:,Idx(plane_azi),:], rect=241, sub=245, swap=True, index=Idx(ths_tran),fig=fig,vmin=0,vmax=VMAX)
            #plot2dtest(Ip.sub[:,Idx(plane_azi),:], rect=242, sub=246, swap=True, index=Idx(ths_tran),fig=fig,vmin=0,vmax=VMAX)
            semi_polar(I.sub[Idx(w_tran),:,:], rect=422, sub=424, swap=False, index=Idx(180-plane_azi),fig=fig,vmin=0,vmax=VMAX)
            semi_polar(Ip.sub[Idx(w_tran),:,:], rect=426, sub=428, swap=False, index=Idx(180-plane_azi),fig=fig,vmin=-0.12,vmax=0.003)
    #else :
        #fig=figure(figsize=(6,6))
        #S = SpecInt2(wi, wb, ex, we, dl, M=M2, field=field, Irradiance=False)
        #semi_polar(S.sub[:,Idx(172),:], rect=211, sub=212, swap=True, index=Idx(40.),fig=fig,vmin=0,vmax=10000)

In [ ]:
aer=AeroOPAC('urban', 0.3, 550.)
atm=Profile('afglms',aer=aer)

In [ ]:
jobp_black = Smartg('SMART-G-PP', wl=550.,# wl = np.linspace(400, 700, 2),
             THVDEG=30, NBPHOTONS=1e9,NBTHETA=90,
             atm=atm,OUTPUT_LAYERS=3,NFAER=10000,NFOCE=10000,
             surf=RoughSurface(SUR=3, WIND=5., NH2O=1.34),
             water=IOP_SPM(1.,NANG=7201),
             overwrite=True)

In [ ]:
jobe = Smartg('SMART-G-SP', wl=400.,# wl = np.linspace(400, 700, 2),
             THVDEG=29.9, NBPHOTONS=1e9,NBTHETA=180,
             atm=atm,OUTPUT_LAYERS=3,NFAER=10000,NFOCE=10000,
             surf=RoughSurface(SUR=3, WIND=5., NH2O=1.34),
             #water=IOP_MM(1.,NANG=7201),env=Environment(ENV=1,ENV_SIZE=10,X0=-9,ALB=0.4),
             overwrite=True)

In [ ]:
fig = figure(figsize=(8,8))
M=jobe.read()
I_up = M['I_up (TOA)']
I_up0= M['I_up (0+)']
Inp_up = M['Q_up (TOA)']
Inp_up.data=I_up.data - np.sqrt(Inp_up.data**2 + M['U_up (TOA)'].data**2)
Inp_up.desc='Inp_up (TOA)'
Inp_up0= M['Q_up (0+)']
Inp_up0.data=I_up0.data - np.sqrt(Ip_up0.data**2 + M['U_up (0+)'].data**2)
Inp_up0.desc='Inp_up (0+)'
semi_polar(I_up, index=Idx(90), rect=421, sub=423, swap=False, fig=fig,vmin=0,vmax=0.5)
semi_polar(Inp_up, index=Idx(90), rect=422, sub=424, swap=False, fig=fig,vmin=0,vmax=0.5)
semi_polar(I_up0, index=Idx(90), rect=425, sub=427, swap=False, fig=fig,vmin=0,vmax=0.05) 
semi_polar(Inp_up0, index=Idx(90), rect=426, sub=428, swap=False, fig=fig,vmin=0,vmax=0.05)

In [ ]:
fig1 = figure(figsize=(8,8))
M=jobp_black.read()
I_up = M['I_up (TOA)']
I_up0= M['I_up (0+)']
Inp_up = M['Q_up (TOA)']
Inp_up.data=M['I_up (TOA)'].data - np.sqrt(M['Q_up (TOA)'].data**2 + M['U_up (TOA)'].data**2)
Inp_up.desc='Inp_up (TOA)'
Inp_up0= M['Q_up (0+)']
Inp_up0.data=M['I_up (0+)'].data - np.sqrt(M['Q_up (0+)'].data**2 + M['U_up (0+)'].data**2)
Inp_up0.desc='Inp_up (0+)'
semi_polar(I_up, index=Idx(120), rect=421, sub=423, swap=False, fig=fig1,vmin=0,vmax=0.2)
semi_polar(Inp_up, index=Idx(120), rect=422, sub=424, swap=False, fig=fig1,vmin=0,vmax=0.2)
semi_polar(I_up0, index=Idx(120), rect=425, sub=427, swap=False, fig=fig1,vmin=0,vmax=0.1) 
semi_polar(Inp_up0, index=Idx(120), rect=426, sub=428, swap=False, fig=fig1,vmin=0,vmax=0.1) 
fig2 = figure(figsize=(8,4))
semi_polar(I_up0/I_up, index=Idx(120), rect=221, sub=223, swap=False, fig=fig2,vmin=0,vmax=1) 
semi_polar(Inp_up0/Inp_up, index=Idx(120), rect=222, sub=224, swap=False, fig=fig2,vmin=0,vmax=1) 
semi_polar(M['I_up (0-)'],vmin=0,vmax=0.1)

In [ ]:
LMIN=400. # nm
LMAX=700. # nm
SAMPLING=1
PFRES=50. # spectral resolution for phase function (nm)
grid='100[20]20[5]10[1]0' # altitude grid for scattering and absorption coefficients
o3=300. # DU
wref=862. # wavelength for AOT and COT reference
aot=0.1
aermodel_l=['maritime_clean','continental_average']
aermodel=aermodel_l[1]
chl=0.1 # mg/l
ws=5. # m/s
aer = AeroOPAC(aermodel, aot, wref)
repname='reptran_solar_msg' # k distribution
surf=RoughSurface(SUR=3, WIND=ws, NH2O=1.34)
wi,wb,we,ex,dl,bands = ReadREPTRAN_bands(repname,LMIN=LMIN,LMAX=LMAX,SAMPLING=SAMPLING,FULL=True)
#pfwav=np.linspace(LMIN, LMAX, num=(LMAX-LMIN)/PFRES+1, endpoint=True)
water=IOP_MM(chl,NANG=7201)
atm=Profile('afglmw',O3=o3
             #,cloud=CloudOPAC('strato',[('wc.sol.mie', 1, 10. , 9., 10.)], 5, 862.)
             ,aer=aer 
           )
job = Smartg('SMART-G-PP', THVDEG=30, wl=bands, NBPHOTONS=1e8,
                    atm=atm,NBLOOP=5000, OUTPUT_LAYERS=3,env=Environment(ENV=1,ENV_SIZE=10.,X0=0.,ALB=0.5),
                    surf=surf, water=water,NFAER=10000,NFOCE=10000,
                    overwrite=True)

In [ ]:
L=job.read()
L.print_info()